InstrumentationKey=bc15d784-6826-402f-8984-e505b080cf18;IngestionEndpoint=https://westeurope-5.in.applicationinsights.azure.com/;LiveEndpoint=https://westeurope.livediagnostics.monitor.azure.com/;ApplicationId=dfaf4a4a-47f2-4043-b83a-1b02a56c1dbc

In [1]:
import streamlit as st
import requests
import time

from azure.monitor.opentelemetry import configure_azure_monitor
from opentelemetry import trace

# Configure la connexion à Application Insights
connection_string = "InstrumentationKey=bc15d784-6826-402f-8984-e505b080cf18;IngestionEndpoint=https://westeurope-5.in.applicationinsights.azure.com/;LiveEndpoint=https://westeurope.livediagnostics.monitor.azure.com/;ApplicationId=dfaf4a4a-47f2-4043-b83a-1b02a56c1dbc"
configure_azure_monitor(connection_string=connection_string)

# Obtenir un tracer
tracer = trace.get_tracer(__name__)

# Configure l'url de l'API sur Azure
domain = "hedredo-sentiment-b4a8fabydkh4hge5.westeurope-01.azurewebsites.net"
st.title("Live Tweet Sentiment Analysis")
text = st.text_input("Enter a tweet please: ")

if st.button("Predict the sentiment"):
    url = f"https://{domain}/predict"
    data = {"tweet": text}
    response = requests.post(url, json=data)

    if response.status_code != 200:
        st.error(
            "Error occurred during prediction. Please verify your input as a text and try again."
        )

    else:
        prediction = response.json()["sentiment"]
        # Display the result
        st.write(f"Tweet entered: {text}")
        st.write("Model computing the sentiment of the tweet...")
        time.sleep(1)
        # Display the result
        match prediction:
            case "positive":
                st.success(f"Sentiment predicted: {prediction}")
            case "negative":
                st.error(f"Sentiment predicted: {prediction}")
            case _:
                st.write(f"Sentiment predicted: {prediction}")

        # Ajoute une demande d'approbation par l'utilisateur de la prédiction
        validation = st.radio("Is the prediction seems correct to you?", ("Yes", "No"))

        # Envoie une trace à Application Insights si l'utilisateur n'approuve la prédiction
        if validation == "No":
            with tracer.start_as_current_span("IncorrectPrediction"):
                trace.get_current_span().add_event(
                    "User indicated an invalid prediction",
                    {"prediction": prediction, "tweet": text},
                )
                st.info(
                    "Thank you for your feedback. It will be used to improve the model."
                )
        else:
            st.info("Thank you for your feedback.")

2024-12-20 11:30:27.035 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-20 11:30:27.121 
  command:

    streamlit run /home/hedredo/miniconda310/envs/light_p7/lib/python3.10/site-packages/ipykernel_launcher.py [ARGUMENTS]
2024-12-20 11:30:27.122 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-20 11:30:27.123 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-20 11:30:27.123 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-20 11:30:27.124 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-20 11:30:27.124 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-20 11:30:27.

In [56]:
from typing import Union
from pydantic import BaseModel
from fastapi import FastAPI
from model import predict_sentiment

app = FastAPI()


# Create a Pydantic model to validate the request body
class TextData(BaseModel):
    text: str


# Define a GET endpoint for the root URL
@app.get("/")
def read_root() -> Union[dict, str]:
    return {"message": "Welcome to the Sentiment Analysis API!"}


@app.post("/predict")
def predict(data: TextData) -> dict:
    text = data.text
    sentiment = predict_sentiment(text)
    return {"sentiment": sentiment}

In [61]:
import re

pos_pattern = r"good|happy|nice|excellent|positive|fortunate|correct|superior|great|positive|superb|wonderful|awesome|fantastic|terrific|amazing|incredible|fabulous|marvelous|excellent|outstanding|exceptional|perfect|pleasing|delightful|pleasurable|satisfying|acceptable|agreeable|enjoyable|favorable|good|gratifying|great|pleasing|positive"
# Compile the regular expression pattern into a regular expression object
pos_re = re.compile(pos_pattern, re.IGNORECASE)
neg_pattern = r"bad|unhappy|horrible|negative|unfortunate|wrong|inferior|poor|negative|dreadful|terrible|awful|atrocious|abysmal|appalling|dreadful|lousy|unsatisfactory|unacceptable|disagreeable|displeasing|unfavorable|unpleasant|bad|disgusting|distasteful|foul|gross|nasty|nauseating|obnoxious|offensive|repellent|repulsive|revolting|vile|wretched|bad|disagree"
neg_re = re.compile(neg_pattern, re.IGNORECASE)


def predict_sentiment(text: str) -> str:
    # Count the number of positive and negative words in the input
    pos_count = len(pos_re.findall(text))
    neg_count = len(re.findall(neg_re, text))
    # Compute the sentiment score
    sentiment_score = pos_count - neg_count
    # Match the sentiment score to a sentiment category
    if sentiment_score > 0:
        label = "positive"
    elif sentiment_score < 0:
        label = "negative"
    else:
        label = "neutral"
    return label

In [32]:
text = "I am very happy with the service. It was excellent."

In [41]:
text = 1

In [42]:
predict_sentiment(text)

TypeError: expected string or bytes-like object